In [2]:
import pickle
import numpy as np

import argparse
import datetime
import collections
import inspect

import logging
import time
import os.path
import sys

import threading

sys.path.insert(0, "/Jts/tws api/source/pythonclient")


from ibapi import wrapper
from ibapi import utils
from ibapi.client import EClient
from ibapi.utils import iswrapper

from ibapi.common import * # @UnusedWildImport
from ibapi.order_condition import * # @UnusedWildImport
from ibapi.contract import * # @UnusedWildImport
from ibapi.order import * # @UnusedWildImport
from ibapi.order_state import * # @UnusedWildImport
from ibapi.execution import Execution
from ibapi.execution import ExecutionFilter
from ibapi.commission_report import CommissionReport
from ibapi.ticktype import * # @UnusedWildImport
from ibapi.tag_value import TagValue

from ibapi.account_summary_tags import *
from datetime import date

In [3]:
window = 60
def fill_zeros(day_dic):
    to_del = []
    for day in day_dic:
        
        count = 0
        for m in range(1, len(day_dic[day])):
            if day_dic[day][m][3] == 0:
                day_dic[day][m][0:4] = day_dic[day][m-1][3]
                count += 1
        for m in range(1, len(day_dic[day])+1):
            if day_dic[day][-m][3] == 0:
                day_dic[day][-m][0:4] = day_dic[day][-(m-1)][3]
                count += 1
        
        if count > 100:
            print(day, count)
            to_del.append(day)
    for day in to_del:
        del day_dic[day]
    return day_dic
        
def ibapi_to_ohlc(fname, mkt_hours = False):
    dat = pickle.load(open(fname, "rb"))
    sym_dic = {}
    for bar in dat:
        ts = int(bar.date)
        sym_dic[ts] = bar.close
    '''
    print(max(list(sym_dic.keys())))
    day_dic = {}
    
    for ts in sym_dic:
        day = str(date.fromtimestamp(ts))
        if "-01-01" in day or "-12-25" in day or "-11-26" in day or "-11-27" in day or "-12-24" in day or "-07-03" in day:
            continue
        if not day in day_dic:
            day_dic[day] = np.zeros((int(390 * 60 / window), 4))
        min_ind = int((ts % (24*3600) - 48600)/window)
        
        for i in range(4):
            if day_dic[day][min_ind,i] == 0:
                day_dic[day][min_ind,i] = sym_dic[ts][i]
        else:
            day_dic[day][min_ind,1] = max(sym_dic[ts][1], day_dic[day][min_ind,1])
            day_dic[day][min_ind,2] = min(sym_dic[ts][2], day_dic[day][min_ind,2])
        if ts % window == 0:
            day_dic[day][min_ind,0] = sym_dic[ts][0]
        if ts % window == window - 10 or day_dic[day][min_ind,3] == 0:
            day_dic[day][min_ind,3] = sym_dic[ts][3]

    day_dic = fill_zeros(day_dic)
    '''
    return sym_dic
    

In [6]:
symbols = ['SPY', 'GME','TSLA','GOOG','AAPL']
all_dat = {}
for sym in symbols:
    print(sym)
    all_dat[sym] = ibapi_to_ohlc("../raw_pricedata_{s}.pkl".format(s=sym))

SPY
GME
TSLA
GOOG
AAPL


In [8]:
dat = pickle.load(open("../raw_pricedata_SPY.pkl", "rb"))

In [9]:
dat

[1886723036448: Date: 1619510400, Open: 418.060000, High: 418.060000, Low: 418.060000, Close: 418.060000, Volume: 3, Average: 418.060000, BarCount: 2,
 1886723061936: Date: 1619510460, Open: 418.060000, High: 418.060000, Low: 418.060000, Close: 418.060000, Volume: 0, Average: 418.060000, BarCount: 0,
 1886723063232: Date: 1619510520, Open: 418.090000, High: 418.090000, Low: 418.090000, Close: 418.090000, Volume: 1, Average: 418.090000, BarCount: 1,
 1886723063424: Date: 1619510580, Open: 418.120000, High: 418.120000, Low: 418.120000, Close: 418.120000, Volume: 1, Average: 418.120000, BarCount: 1,
 1886723059968: Date: 1619510640, Open: 418.120000, High: 418.120000, Low: 418.120000, Close: 418.120000, Volume: 0, Average: 418.120000, BarCount: 0,
 1886723063760: Date: 1619510700, Open: 418.180000, High: 418.180000, Low: 418.180000, Close: 418.180000, Volume: 5, Average: 418.180000, BarCount: 5,
 1886723061696: Date: 1619510760, Open: 418.270000, High: 418.270000, Low: 418.270000, Close: 

In [7]:
for sym in symbols:
    f = open("../processed_{:s}.csv".format(sym), "wt")
    csv_string = ""
    for ts in sorted(all_dat[sym]):
        csv_string += str(ts) + ", " + str(date.fromtimestamp(ts)) + ", " + str(all_dat[sym][ts]) + "\n"
    f.write(csv_string)
    f.close()

In [22]:
all_dat['DIA']

{1613568600: 315.75,
 1613568660: 315.41,
 1613568720: 315.43,
 1613568780: 315.58,
 1613568840: 315.55,
 1613568900: 315.55,
 1613568960: 315.53,
 1613569020: 315.53,
 1613569080: 315.76,
 1613569140: 315.74,
 1613569200: 315.74,
 1613569260: 315.74,
 1613569320: 315.74,
 1613569380: 315.67,
 1613569440: 315.64,
 1613569500: 315.54,
 1613569560: 315.43,
 1613569620: 315.43,
 1613569680: 315.43,
 1613569740: 315.43,
 1613569800: 315.43,
 1613569860: 315.43,
 1613569920: 314.83,
 1613569980: 315.03,
 1613570040: 314.98,
 1613570100: 315.0,
 1613570160: 315.0,
 1613570220: 315.21,
 1613570280: 315.21,
 1613570340: 315.21,
 1613570400: 314.98,
 1613570460: 314.98,
 1613570520: 314.98,
 1613570580: 314.98,
 1613570640: 315.26,
 1613570700: 315.26,
 1613570760: 315.26,
 1613570820: 315.26,
 1613570880: 314.86,
 1613570940: 314.82,
 1613571000: 315.01,
 1613571060: 315.01,
 1613571120: 314.75,
 1613571180: 314.69,
 1613571240: 314.73,
 1613571300: 314.73,
 1613571360: 314.73,
 1613571420: 31